In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [24]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display, Math
from six.moves import cPickle as pickle
import string
from PIL import Image
import numpy as np
import h5py
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [25]:
width = None
pd.options.display.max_rows = 600
pd.options.display.max_columns = width
pd.options.display.max_colwidth = 600
pd.options.display.width = width
pd.options.display.max_seq_items = None
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [26]:
import data_commons as dtc
import dl_commons as dlc
import viz_commons as vc
from viz_commons import VisualizeDir, DiffParams, VisualizeStep

In [27]:
vd = VisualizeDir(os.path.expanduser('~/im2latex/src/tb_metrics_dev/2017-11-17 21-30-42 PST newData_3.1LSTM_2init_3OutMLP_3attMLPfull_1beta_0pLambda_regroupImage_i=2/store'))

In [28]:
vd.view_steps()

num_epoch_steps = 25
epoch_steps = [1509, 3018, 4527, 6036, 7545, 9054, 10563, 12072, 13581, 15090, 16599, 18108, 19617, 21126, 22635, 24144, 25653, 27162, 28671, 30180, 31689, 33198, 34707, 36216, 37725]


In [14]:
vd.hyper.pLambda

0.0

In [29]:
vs2 = VisualizeStep(vd, 'training', 27162)
vs2.keys()

[u'alpha', u'beta', u'bleu', u'ed', u'image_name', u'predicted_ids', u'y']

In [34]:
vs2.nd('beta').shape

(1, 56, 61)

In [30]:
vs2.strs('predicted_ids')

,edit_distance,predicted_ids
47,0.017544,"a_{-}^{\prime }-\frac {1}{\rho }a_{-}=\frac {\eta ^{2}\rho }{e}\left( p_{6}^{2}-\frac {1-c}{2}\right) ,\eos -2"
52,0.038462,q^{\prime }=\frac {4\epsilon }{a^{2}(d-1)(d-2)\mathcal \cal H}({\cal H}^{2}-{\cal H}^{\prime }).\eos \eos \eos \eos \eos \eos \eos -2
12,0.050847,\hat {H}=\lambda (A^{2}+D^{2})+e^{i\pi (P-1)/2}q^{-1/2}(q-q^{-1})(C+B)\eos \eos \eos
23,0.053571,(e\cdot D_{j})=\sum _{n=1}^{2}(e-P_{n})(P_{n}\rightarrow Q_{n})(Q_{n}\rightarrow D_{n})\eos \eos \eos
45,0.054545,\varphi _{1}^{\prime }(0_{+})=\varphi _{-}(0_{-})=\varphi _{+}^{\prime }(t)=\varphi _{-}(-t)=0.\eos \eos \eos -2
8,0.067797,"\gamma _{n}^{\mu }=\sqrt {4nx}\, c_{n}^{\mu },\quad ~\gamma _{-n}^{\mu }=\sqrt {4nx}\, \epsilon _{n}^{\mu },\eos \eos"
42,0.075472,"K_{\mu }(u)=\frac {2\int _{0}^{\infty }d\tau \, \tau ^{\mu }1}\operatorname {exp}\left[ -\frac {2(\tau +\tau ^{-1})\right] \eos \eos \eos \eos \eos -2"
49,0.076923,"S_{x}+S_{x}{\prime }=\frac {1}{32}W^{2}(x|^{2}+|x^{\prime }|^{2}),\eos \eos \eos \eos \eos \eos \eos \eos \eos -2"
35,0.078431,"\{ \phi _{\alpha },H_{E}\} =\{ \phi _{\alpha },V\} +\sum _{\beta }\, \phi _{\alpha },\phi _{\beta }\} =_{\beta }=0.\eos \eos \eos \eos \eos \eos \eos \eos -2"
46,0.084746,\frac {dm^{2}(t)}{dt}=\frac {2\alpha ^{2}\eta ^{2}(t)}{Q^{2}+\Delta _{mn{\prime }m^{2}(t)\eos -2


In [32]:
nd_alpha = vs2.alpha(invert_alpha=True)

CustomConvnetImageProcessor: image_batch_shape = (1, 130, 1090, 1), image_shape = [130, 1090, 1]


<IPython.core.display.Math object>

a_{-}^{\prime }-\frac {1}{\rho }a_{-}=\frac {\eta ^{2}\rho }{e}\left( p_{6}^{2}-\frac {1-c}{2}\right) ,


<IPython.core.display.Math object>

a_{-}^{\prime }-\frac {1}{\rho }a_{-}=\frac {\eta ^{2}\rho }{e}\left( \psi _{6}^{2}-\frac {1-c}{2}\right) ,
